https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html

In [11]:
from datasets import load_dataset

In [12]:
dataset = load_dataset('rotten_tomatoes')

Found cached dataset rotten_tomatoes (C:/Users/Siddhi/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
100%|██████████| 3/3 [00:00<00:00, 301.40it/s]


In [13]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [27]:
dataset["train"][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [14]:
train_data = load_dataset('rotten_tomatoes',split='train')

test_data = load_dataset('rotten_tomatoes',split='test')

validation_data = load_dataset('rotten_tomatoes',split='validation')

Found cached dataset rotten_tomatoes (C:/Users/Siddhi/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (C:/Users/Siddhi/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (C:/Users/Siddhi/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


In [15]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

In [16]:
len(train_data)

8530

In [17]:
train_data[0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [21]:
train_data.shape

(8530, 2)